# Second exercise: top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.


In [ ]:
Load and clean data: remove the first line and the delimiter

In [69]:
val bookings = sc.textFile("bookings.csv.bz2")
val cleandata = bookings.map(line => line.split("\\^").map(_.trim))
val firstRow = cleandata.first


Remove the first line of the file (header) in this inline function can fail.
Spark can complain the "NotSerializableException occurs in Scala" it means that a reference to firstRow:Array[String] is not serializable. 
It's still not clear to me but I think is a Spark environment configuration problem

In the second line there is the alternative version always working just passing the literal of the first string in firstRow array. This version can't be used in general case to remove the first line of a file loaded in a RDD.

Anyway I am sure or at least I excpect there is a more efficient way to remove the first line of a file loaded in a RDD in Spark

In [79]:
val data = cleandata.filter( line => !line.contains(firstRow(0)) )

In [81]:
val data = cleandata.filter( line => !line.contains("act_date") )

check if the first line is correct

In [82]:
data.first

Array(2013-03-05 00:00:00, 1A, DE, a68dd7ae953c8acfb187a1af2dcbe123, 1a11ae49fcbf545fd2afc1a24d88d2b7, ea65900e72d71f4626378e2ebd298267, 2013-02-22 00:00:00, 1708, 0, ZRH, ZRH, CH, LHR, LON, GB, ZRH, ZRH, CH, LHR, LON, GB, ZRH, ZRH, CH, LHRZRH, LONZRH, CHGB, 1, LHRZRH, VI, T, Y, 2013-03-07 08:50:00, 2013-03-07 11:33:37, -1, 2013, 3, NULL)

check if the number of lines are correct ( bookings.count - 1 )

In [83]:
data.count

10000010

processing the lines I got error in reduceByKey() function (java.lang.ArrayIndexOutOfBoundsException)
It might be that there are some lines empty or with not enough elements

In [91]:
 val numberFields = firstRow.length   // number of columns excpected for each line
 val filteredData = data.filter( line => line.length >= numberFields) 
 filteredData.count

10000009

Create a Key Value Pair RDD where key=arr_port  value=pax

In [86]:
import org.apache.spark.storage.StorageLevel
 
val arr_port = firstRow.indexOf("arr_port") // index of arrival airport column in csv data file
val pax = firstRow.indexOf("pax")           // index of number of passangers column in csv data file

val keyValRDD = filteredData.map{ x => (x(arr_port), x(pax).toLong ) }


In [ ]:
Sum all the values (pax) with the same key (arr_port) and store in keyValReduced

In [87]:
val keyValReduced = keyValRDD.reduceByKey( (x,y) => x + y )
//keyValReduced.persist(StorageLevel.MEMORY_ONLY)
keyValReduced.count

2274

compute the first 10 arrival just reordering by value the keyValReduced RDD and print the 10 firsts Key Value Pair.
it's safe to run foreach because sortedRDD.take(..) move the RDD data on the Driver Node

In [3]:
println("Results:\n")
val sortedRDD = keyValReduced.sortBy(  _._2 , false)
sortedRDD.take(10).foreach(println)

Results:

(LHR,88809)
(MCO,70930)
(LAX,70530)
(LAS,69630)
(JFK,66270)
(CDG,64490)
(BKK,59460)
(MIA,58150)
(SFO,58000)
(DXB,55590)
